In [ ]:
import argparse
import os
import json

from text2sql import _test

batch_size = 8
device = "2" #'the id of used GPU device.')
seed = 42,
save_path = "./models/text2sql" #'save path of fine-tuned text2sql models.')
eval_results_path = "./eval_results/text2sql"
mode = "eval"
dev_filepath = "./data/pre-processing/resdsql_test.json"
original_dev_filepath = "./data/spider/dev.json"
db_path = "./data/spider/database"
num_beams = 8 #'beam size in model.generate() function.')
num_return_sequences = 8 #'the number of returned sequences in model.generate() function (num_return_sequences <= num_beams).')
output = predicted_sql.txt

In [ ]:
ckpt_names = os.listdir(save_path)
ckpt_names = sorted(ckpt_names, key = lambda x:eval(x.split("-")[1]))

print("ckpt_names:", ckpt_names)

save_path = save_path
os.makedirs(eval_results_path, exist_ok = True)

eval_results = []
for ckpt_name in ckpt_names:
    
    # if the current ckpt is being evaluated or has already been evaluated
    if "{}.txt".format(ckpt_name) in os.listdir(eval_results_path):
        
        # is being evaluated
        with open(eval_results_path+"/{}.txt".format(ckpt_name), "r") as f:
            if len(f.readlines()) == 1:
                continue
                
        # has already been evaluated
        with open(eval_results_path+"/{}.txt".format(ckpt_name), "r") as f:
            eval_result = json.load(f)
            eval_results.append(eval_result)
            
    # otherwise, we start evaluating the current ckpt
    else:
        print("Start evaluating ckpt: {}".format(ckpt_name))
        with open(eval_results_path+"/{}.txt".format(ckpt_name), "w") as f:
            f.write("Evaluating...")

        save_path = save_path + "/{}".format(ckpt_name)
        em, exec = _test(opt)

        eval_result = dict()
        eval_result["ckpt"] = save_path
        eval_result["EM"] = em
        eval_result["EXEC"] = exec

        with open(eval_results_path+"/{}.txt".format(ckpt_name), "w") as f:
            f.write(json.dumps(eval_result, indent=2))

        eval_results.append(eval_result)

In [ ]:
for eval_result in eval_results:
    print("ckpt name:", eval_result["ckpt"])
    print("EM:", eval_result["EM"])
    print("EXEC:", eval_result["EXEC"])
    print("-----------")

em_list = [er["EM"] for er in eval_results]
exec_list = [er["EXEC"] for er in eval_results]
em_and_exec_list = [em + exec for em, exec in zip(em_list, exec_list)]

# find best EM ckpt
best_em, exec_in_best_em = 0.00, 0.00
best_em_idx = 0

# find best EXEC ckpt
best_exec, em_in_best_exec = 0.00, 0.00
best_exec_idx = 0

# find best EM + EXEC ckpt
best_em_plus_exec = 0.00
best_em_plus_exec_idx = 0

for idx, (em, exec) in enumerate(zip(em_list, exec_list)):
    if em > best_em or (em == best_em and exec > exec_in_best_em):
        best_em = em
        exec_in_best_em = exec
        best_em_idx = idx

    if exec > best_exec or (exec == best_exec and em > em_in_best_exec):
        best_exec = exec
        em_in_best_exec = em
        best_exec_idx = idx

    if em+exec > best_em_plus_exec:
        best_em_plus_exec = em+exec
        best_em_plus_exec_idx = idx

print("Best EM ckpt:", eval_results[best_em_idx])
print("Best EXEC ckpt:", eval_results[best_exec_idx])
print("Best EM+EXEC ckpt:", eval_results[best_em_plus_exec_idx])